In [113]:
#import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pylab as pl
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import MiniBatchKMeans
from preprocess import *
from collections import Counter
import cv2
from skimage.transform import pyramid_reduce
from sklearn.metrics import accuracy_score
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
A = load_Dataset("../fonts-dataset/Scheherazade New/*.jpeg")
B= load_Dataset("../fonts-dataset/Lemonada/*.jpeg")
C= load_Dataset("../fonts-dataset/Marhey/*.jpeg")
D= load_Dataset("../fonts-dataset/IBM Plex Sans Arabic/*.jpeg")

400
400
400
400


In [115]:
A, A_TEST = train_test_split(A, test_size=0.2)
B, B_TEST = train_test_split(B, test_size=0.2)
C, C_TEST = train_test_split(C, test_size=0.2)
D, D_TEST = train_test_split(D, test_size=0.2)

In [116]:
A_PROCESSED = []
for img in A:
    img=denoise(img)
    A_PROCESSED.append(img)
print("A: ", len(A_PROCESSED))
B_PROCESSED = []
for img in B:
    img=denoise(img)
    B_PROCESSED.append(img)
print("B: ", len(B_PROCESSED))
C_PROCESSED = []
for img in C:
    img=denoise(img)
    C_PROCESSED.append(img)
print("C: ", len(C_PROCESSED))
D_PROCESSED = []
for img in D:
    img=denoise(img)
    D_PROCESSED.append(img)
print("D: ", len(D_PROCESSED))

A:  320
B:  320
C:  320
D:  320


In [117]:
def get_level(image):
    img=image.copy()
    levels = []
    while img.shape[0] > 64:
        levels.append(img)
        img = pyramid_reduce(img)
    levels.append(img)
    return levels

In [118]:
def get_levels(images):
    levels = []
    for img in images:
        levels.append(get_level(img))
    return levels

In [119]:
def get_gaussian_kernel(m=4):
    x_index, y_index = np.meshgrid(np.arange(-m, m + 1), np.arange(-m, m + 1))
    x_index = x_index.reshape((-1, 1))
    y_index = y_index.reshape((-1, 1))

    G = lambda x, y: np.exp(-(x**2 + y**2))
    G0 = lambda x, y: -2*x*G(x,y)

    # OPTIONAL: remove negative (to align mathematically) since vertical numers increase as we go down 
    G90 = lambda x, y: 2*y*G(x,y) 

    # Gaussian Kernel
    gk = G(x_index, y_index).reshape((m*2 + 1, m*2 + 1))

    # Gaussian 1st Deriviative with 0 degree orientation
    gk0 = G0(x_index, y_index).reshape((m*2 + 1, m*2 + 1))

    # Gaussian 1st Deriviative with 90 degree orientation
    gk90 = G90(x_index, y_index).reshape((m*2 + 1, m*2 + 1))
    return gk0, gk90

In [120]:
def get_steering_filters(steering_angles,gk0,gk90):
    gk_theta=[]
    for angle in steering_angles:
        gk_theta.append(np.cos(np.radians(angle))*gk0 + np.sin(np.radians(angle))*gk90)
    return gk_theta
    

In [121]:
def apply_filters_get_variance(levels,filters):
    variances = []
    for level in levels:
        for filteri in filters:
            filtered=cv2.filter2D(level, -1, filteri)
            variances.append(np.var(filtered))
    return variances

In [122]:
gk0, gk90 = get_gaussian_kernel()
steering_angles = np.arange(0, 180, 30)
gk_thetas = get_steering_filters(steering_angles, gk0, gk90)

In [123]:
A_levels=[]
A_levels=(get_levels(A_PROCESSED))
B_levels=[]
B_levels=(get_levels(A_PROCESSED))
C_levels=[]
C_levels=(get_levels(A_PROCESSED))
D_levels=[]
D_levels=(get_levels(A_PROCESSED))

In [124]:
A_Vars=[]
for levels in A_levels:
    A_Vars.append(apply_filters_get_variance(levels,gk_thetas))
B_Vars=[]
for levels in B_levels:
    B_Vars.append(apply_filters_get_variance(levels,gk_thetas))
C_Vars=[]
for levels in C_levels:
    C_Vars.append(apply_filters_get_variance(levels,gk_thetas))
D_Vars=[]
for levels in D_levels:
    D_Vars.append(apply_filters_get_variance(levels,gk_thetas))

In [125]:
train_labels=np.concatenate((np.zeros(len(A_Vars)),np.ones(len(B_Vars)),np.full(len(C_Vars),2),np.full(len(D_Vars),3)))

In [126]:
combined_list=A_Vars+B_Vars+C_Vars+D_Vars

In [127]:
def train_knn_classifier(k, train_features, train_labels):
    # Create KNN classifier object
    knn_classifier = KNeighborsClassifier(n_neighbors=k,n_jobs=-1,weights='distance',metric='euclidean',algorithm='auto')
    
    # Train the classifier
    knn_classifier.fit(train_features, train_labels)
    
    return knn_classifier

In [128]:
knn_classifier = train_knn_classifier(11,combined_list,train_labels)

In [129]:
A_TLEVELS = []
for img in A_TEST:
    img=denoise(img)
    A_TLEVELS.append(img)
print("A: ", len(A_TLEVELS))
B_TLEVELS = []
for img in B_TEST:
    img=denoise(img)
    B_TLEVELS.append(img)
print("B: ", len(B_TLEVELS))
C_TLEVELS = []
for img in C_TEST:
    img=denoise(img)
    C_TLEVELS.append(img)
print("C: ", len(C_TLEVELS))
D_TLEVELS = []
for img in D_TEST:
    img=denoise(img)
    D_TLEVELS.append(img)
print("D: ", len(D_TLEVELS))

A:  80
B:  80
C:  80
D:  80


In [130]:
A_TLEVELS=get_levels(A_TLEVELS)
B_TLEVELS=get_levels(B_TLEVELS)
C_TLEVELS=get_levels(C_TLEVELS)
D_TLEVELS=get_levels(D_TLEVELS)

In [131]:
A_TVars=[]
for levels in A_TLEVELS:
    A_TVars.append(apply_filters_get_variance(levels,gk_thetas))
B_TVars=[]
for levels in B_TLEVELS:
    B_TVars.append(apply_filters_get_variance(levels,gk_thetas))
C_TVars=[]
for levels in C_TLEVELS:
    C_TVars.append(apply_filters_get_variance(levels,gk_thetas))
D_TVars=[]
for levels in D_TLEVELS:
    D_TVars.append(apply_filters_get_variance(levels,gk_thetas))

In [132]:
combined_test_list=A_TVars+B_TVars+C_TVars+D_TVars
test_labels=np.concatenate((np.zeros(len(A_TVars)),np.ones(len(B_TVars)),np.full(len(C_TVars),2),np.full(len(D_TVars),3)))

In [133]:
predictions = knn_classifier.predict(combined_test_list)
accuracy = accuracy_score(test_labels, predictions)

print("Accuracy:", accuracy*100)

Accuracy: 24.0625
